# [실습]서울시 구별 CCTV 현황 분석
- CCTV가 어느 구에 가장 많이 설치되었는지 알아보기
- 구별 인구 대비 CCTV 설치 비율 알아보기 
- 구별 CCTV 현황 시각화해보기

사용하는 데이터
- 서울 열린데이터 광장에서 서울시 구별 CCTV 설치 현황데이터(서울시CCTV설치운영현황(자치구)_년도별_210731기준.csv)
- 서울 열린데이터 광장에서 서울시 주민등록인구 (구별) 통계데이터

In [1]:
# 필요한 libary import

import pandas as pd
import numpy as np

In [46]:
# 서울시 구별 CCTV 설치 현황데이터 불러오기
CCTV_seoul= pd.read_csv('CCTV_in_seoul.csv', encoding='cp949',header=1)
CCTV_seoul.head()

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,"77,032","7,667","2,200","3,491","4,439","6,582","8,129","9,947","9,876","11,961","11,132","1,608"
1,종로구,"1,772",813,0,0,210,150,1,261,85,9,200,43
2,중 구,"2,333",16,114,87,77,236,240,372,386,155,361,289
3,용산구,"2,383",34,71,234,125,221,298,351,125,307,617,0
4,성동구,"3,602",448,125,212,105,339,310,874,390,262,461,76


In [47]:
# 구분을 구별로 이름 변경
CCTV_seoul.rename(columns={'구분':'구별'},inplace=True)
CCTV_seoul.head()

,구별,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,"77,032","7,667","2,200","3,491","4,439","6,582","8,129","9,947","9,876","11,961","11,132","1,608"
1,종로구,"1,772",813,0,0,210,150,1,261,85,9,200,43
2,중 구,"2,333",16,114,87,77,236,240,372,386,155,361,289
3,용산구,"2,383",34,71,234,125,221,298,351,125,307,617,0
4,성동구,"3,602",448,125,212,105,339,310,874,390,262,461,76


In [68]:
# 서울시 인구 데이터 불러오기 
# 필요한 칼럼만 가져오기
pop_seoul=pd.read_excel('population_in_seoul.xls',header=2,
                       usecols='B,D,G,J,N')
pop_seoul.head()

,자치구,계,계.1,계.2,65세이상고령자
0,합계,9765869,9542256,223613,1592339
1,종로구,154318,145512,8806,27776
2,중구,131943,122858,9085,24225
3,용산구,238300,224145,14155,38906
4,성동구,294140,287820,6320,46053


In [69]:
# 칼럼이름 변경
pop_seoul.rename(columns={'자치구':'구별',
                          '계':'인구수',
                         '계.1':'한국인',
                         '계.2':'외국인',
                         '65세이상고령자':'고령자'},inplace=True)
pop_seoul.head()

,구별,인구수,한국인,외국인,고령자
0,합계,9765869,9542256,223613,1592339
1,종로구,154318,145512,8806,27776
2,중구,131943,122858,9085,24225
3,용산구,238300,224145,14155,38906
4,성동구,294140,287820,6320,46053


## 데이터 탐색 - CCTV 현황데이터

In [48]:
# CCTV 개수가 가장 많은 구 top5
CCTV_seoul.sort_values(by='총계',ascending=True).head()

,구별,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
10,도봉구,"1,629",39,22,96,181,79,159,134,222,198,168,331
1,종로구,"1,772",813,0,0,210,150,1,261,85,9,200,43
20,동작구,"2,297",41,24,25,503,128,253,271,300,322,419,11
2,중 구,"2,333",16,114,87,77,236,240,372,386,155,361,289
18,금천구,"2,374",0,0,178,80,361,133,196,540,369,508,9


숫자형이 아니라 원하던 대로 정렬이 되지 않았음 , 숫자형으로 변경해줘야 함

In [49]:
CCTV_seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   구별        26 non-null     object
 1   총계        26 non-null     object
 2   2012년 이전  26 non-null     object
 3   2012년     26 non-null     object
 4   2013년     26 non-null     object
 5   2014년     26 non-null     object
 6   2015년     26 non-null     object
 7   2016년     26 non-null     object
 8   2017년     26 non-null     object
 9   2018년     26 non-null     object
 10  2019년     26 non-null     object
 11  2020년     26 non-null     object
 12  2021년     26 non-null     object
dtypes: object(13)
memory usage: 2.8+ KB


In [56]:
# 자리수 구분자인 , 제거
for i in range(1,13):
    CCTV_seoul[CCTV_seoul.columns[i]]=CCTV_seoul[CCTV_seoul.columns[i]].str.replace(',','') # 총계~2021년까지 , 제거

CCTV_seoul.head()

,구별,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,77032,7667,2200,3491,4439,6582,8129,9947,9876,11961,11132,1608
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43
2,중 구,2333,16,114,87,77,236,240,372,386,155,361,289
3,용산구,2383,34,71,234,125,221,298,351,125,307,617,0
4,성동구,3602,448,125,212,105,339,310,874,390,262,461,76


In [57]:
# 구별을 제외하고 나머지 모두 숫자형으로 변환

for i in range(1,13):
    CCTV_seoul[CCTV_seoul.columns[i]]=CCTV_seoul[CCTV_seoul.columns[i]].astype(int) # 총계~2021년까지 숫자형으로 변환
CCTV_seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   구별        26 non-null     object
 1   총계        26 non-null     int32 
 2   2012년 이전  26 non-null     int32 
 3   2012년     26 non-null     int32 
 4   2013년     26 non-null     int32 
 5   2014년     26 non-null     int32 
 6   2015년     26 non-null     int32 
 7   2016년     26 non-null     int32 
 8   2017년     26 non-null     int32 
 9   2018년     26 non-null     int32 
 10  2019년     26 non-null     int32 
 11  2020년     26 non-null     int32 
 12  2021년     26 non-null     int32 
dtypes: int32(12), object(1)
memory usage: 1.5+ KB


In [60]:
# CCTV 개수가 가장 많은 구 top5
CCTV_seoul.sort_values(by='총계',ascending=False).head(6)

,구별,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,계,77032,7667,2200,3491,4439,6582,8129,9947,9876,11961,11132,1608
23,강남구,6502,124,77,75,597,840,1310,999,748,789,942,1
21,관악구,4942,428,205,291,513,529,621,687,663,640,331,34
17,구로구,4075,852,216,349,187,268,326,540,488,434,415,0
8,성북구,3958,83,78,170,230,323,594,460,867,714,251,188
12,은평구,3791,14,3,44,332,329,555,403,635,1057,288,131


CCTV 개수가 가장 많은 구는 강남구, 관악구, 구로구, 성북구, 은평구 순이다.

In [62]:
# CCTV 개수가 가장 적은 구 top5
CCTV_seoul.sort_values(by='총계',ascending=True).head()

,구별,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
10,도봉구,1629,39,22,96,181,79,159,134,222,198,168,331
1,종로구,1772,813,0,0,210,150,1,261,85,9,200,43
20,동작구,2297,41,24,25,503,128,253,271,300,322,419,11
2,중 구,2333,16,114,87,77,236,240,372,386,155,361,289
18,금천구,2374,0,0,178,80,361,133,196,540,369,508,9


CCTV 개수가 가장 많은 구는 도봉구, 종로구, 동작구, 중구, 금천구 순이다.

In [63]:
### CCTV 최근 3년간 증가율 구하기
CCTV_seoul['최근증가율']=(CCTV_seoul['2021년']+CCTV_seoul['2020년']+CCTV_seoul['2019년'])/(CCTV_seoul['2012년 이전']+CCTV_seoul['2012년']+CCTV_seoul['2013년']+CCTV_seoul['2014년']+CCTV_seoul['2015년']+CCTV_seoul['2016년']+CCTV_seoul['2017년']+CCTV_seoul['2018년'])*100
CCTV_seoul.sort_values(by='최근증가율',ascending=False).head()

,구별,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,최근증가율
9,강북구,2462,0,0,24,65,105,243,6,392,1000,588,39,194.850299
7,중랑구,3296,302,24,253,88,141,161,162,173,1049,939,4,152.760736
24,송파구,2854,72,61,86,85,215,148,241,542,1068,235,101,96.827586
10,도봉구,1629,39,22,96,181,79,159,134,222,198,168,331,74.785408
12,은평구,3791,14,3,44,332,329,555,403,635,1057,288,131,63.758099


최근 3년간 CCTV가 가장 많이 증가한 구는 강북구, 중랑구, 송파구, 도봉구, 은평구 순이다.

## 데이터 탐색 - 서울시 인구현황데이터

In [70]:
pop_seoul.head()

,구별,인구수,한국인,외국인,고령자
0,합계,9765869,9542256,223613,1592339
1,종로구,154318,145512,8806,27776
2,중구,131943,122858,9085,24225
3,용산구,238300,224145,14155,38906
4,성동구,294140,287820,6320,46053


In [71]:
# 합계행 제거하기
pop_seoul.drop([0],inplace=True)
pop_seoul.head()

,구별,인구수,한국인,외국인,고령자
1,종로구,154318,145512,8806,27776
2,중구,131943,122858,9085,24225
3,용산구,238300,224145,14155,38906
4,성동구,294140,287820,6320,46053
5,광진구,353380,341112,12268,51278


In [72]:
pop_seoul['구별'].unique()

array(['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
       '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구'], dtype=object)

In [73]:
pop_seoul.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 1 to 25
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   구별      25 non-null     object
 1   인구수     25 non-null     int64 
 2   한국인     25 non-null     int64 
 3   외국인     25 non-null     int64 
 4   고령자     25 non-null     int64 
dtypes: int64(4), object(1)
memory usage: 1.2+ KB


In [74]:
# 구별 외국인 비율 & 구별 고령자비율 계산하기
pop_seoul['외국인비율']=pop_seoul['외국인']/pop_seoul['인구수']*100
pop_seoul['고령자비율']=pop_seoul['고령자']/pop_seoul['인구수']*100
pop_seoul.head()

,구별,인구수,한국인,외국인,고령자,외국인비율,고령자비율
1,종로구,154318,145512,8806,27776,5.706398,17.999196
2,중구,131943,122858,9085,24225,6.885549,18.360201
3,용산구,238300,224145,14155,38906,5.939992,16.326479
4,성동구,294140,287820,6320,46053,2.148637,15.656830
5,광진구,353380,341112,12268,51278,3.471617,14.510725


In [76]:
# 인구수가 가장 많은 구 top5
pop_seoul.sort_values(by='인구수',ascending=False).head()

,구별,인구수,한국인,외국인,고령자,외국인비율,고령자비율
24,송파구,664996,659385,5611,96453,0.843764,14.504298
16,강서구,581265,575875,5390,91505,0.927288,15.742389
23,강남구,538075,533359,4716,77222,0.876458,14.351531
11,노원구,517038,513053,3985,87537,0.770736,16.930477
21,관악구,501572,488029,13543,79444,2.700111,15.839002


인구수가 가장 많은 구는 송파구, 강서구, 강남구, 노원구, 관악구 순이다.

In [77]:
# 외국인 비율이 가장 높은 구 top5
pop_seoul.sort_values(by='외국인비율',ascending=False).head()

,구별,인구수,한국인,외국인,고령자,외국인비율,고령자비율
2,중구,131943,122858,9085,24225,6.885549,18.360201
19,영등포구,401814,377536,24278,62108,6.042099,15.456903
3,용산구,238300,224145,14155,38906,5.939992,16.326479
17,구로구,422361,397506,24855,71930,5.884776,17.030455
18,금천구,244887,230644,14243,40691,5.816152,16.616235


외국인 비율이 가장 높은 구는 중구, 영등포구, 용산구, 구로구, 금천구 순이다.

In [78]:
# 고령자 비율이 가장 높은 구 top5
pop_seoul.sort_values(by='고령자비율',ascending=False).head()

,구별,인구수,한국인,외국인,고령자,외국인비율,고령자비율
9,강북구,304257,300887,3370,63914,1.107616,21.006583
10,도봉구,320711,318718,1993,63448,0.621432,19.783543
2,중구,131943,122858,9085,24225,6.885549,18.360201
12,은평구,477961,474062,3899,86724,0.815757,18.144577
7,중랑구,393149,388554,4595,70908,1.168768,18.035910


외국인 비율이 가장 높은 구는 강북구, 도봉구, 중구, 은평구, 중랑구 순이다.